In [3]:
import re
import json
import time
import threading
import numpy as np
import pandas as pd
from tqdm import tqdm
from pprint import pprint
from bs4 import BeautifulSoup
from selenium import webdriver
from geopy.geocoders import Nominatim
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin

In [4]:
with open('./Samples/police_nationale.json', encoding='UTF-8') as f: 
    data = json.load(f)

In [24]:
# Scrapping

comments_list = []
dates_list = []
name_service = []
address_service = []
region_list = []

for administration in data.keys():
    for link in range(len(data[administration])):
        # initialiser le webdriver
        driver = webdriver.Chrome()
        driver.get(data[administration][link])

        # refuser les cookies
        try:
            driver.find_element(By.CLASS_NAME, 'VfPpkd-LgbsSe.VfPpkd-LgbsSe-OWXEXe-k8QpJ.VfPpkd-LgbsSe-OWXEXe-dgl2Hf.nCP5yc.AjY5Oe.DuMIQc.LQeN7.Nc7WLe').click()
            time.sleep(1)
        except:
            continue
        
        # cliquer sur le bouton avis
        try:
            driver.find_elements(By.CLASS_NAME, 'hh2c6')[1].click()
            time.sleep(1)
        except:
            continue

        # faire défiler la page pour charger tous les commentaires
        try:
            scrollable_div = driver.find_element(By.CSS_SELECTOR, "div.m6QErb.DxyBCb.kA9KIf.dS8AEf")
        except:
            continue
        
        old_freq = 0
        while True:
            driver.execute_script(
                'arguments[0].scrollTop = arguments[0].scrollHeight',
                scrollable_div
            )
            time.sleep(3)
            freq = scrollable_div.text.count('Partager\n')
            if freq == old_freq:
                break
            old_freq = freq
        
        try:
            driver.find_element(By.CLASS_NAME,'w8nwRe.kyuRq').click()
        except:
            continue
        
        time.sleep(3)
        
        try:
            comments = driver.find_elements(By.CSS_SELECTOR, 'div.GHT2ce > div > div.MyEned > span.wiI7pd')
        except:
            continue
        
        comments_list.append([comments[elt].text for elt in range(len(comments))])
        
        for indice in range(len(comments)):
            region_list.append(administration.split("région")[-1])
        
        try:
            dates = driver.find_elements(By.CLASS_NAME,'rsqaWe')
        except:
            continue
        
        dates_list.append([dates[i].text for i in range(len(comments))])
        
        try:
            driver.find_element(By.CLASS_NAME,'Gpq6kf.fontTitleSmall').click()
        except:
            continue
        
        time.sleep(3)
        
        try:
            name_service.append(driver.find_element(By.CLASS_NAME,'DUwDvf.fontHeadlineLarge').accessible_name)
            address_service.append(driver.find_element(By.CLASS_NAME,'Io6YTe.fontBodyMedium').text)
        except:
            continue
        
        driver.close()

In [25]:
# Récupérer les listes de services et d'adresses

name_service_list = []
address_service_list = []

for index in range(len(comments_list)):
    for elt in range(len(comments_list[index])):
        name_service_list.append(name_service[index])
        address_service_list.append(address_service[index])

In [26]:
# Récupérer la liste des commentaires
new_comments_list = []
for index in range(len(comments_list)):
    for elt in comments_list[index]:
        new_comments_list.append(elt)
        
# Récupérer la liste des dates
new_dates_list = []
for index in range(len(dates_list)):
    for elt in dates_list[index]:
        new_dates_list.append(elt)

In [27]:
# Création du DataFrame des données

dict_text = {
    'Administration': name_service_list,
    'Avis': new_comments_list,
    'Adresses': address_service_list,
    'Régions': region_list,
    'Dates': new_dates_list
}

df = pd.DataFrame(dict_text)

In [30]:
df.to_csv("./Samples/dataset_police_nationale.csv", sep=',', encoding='UTF-8', header=True, index=False)

In [211]:
# Fonction de récupération des coordonnées géographiques des régions
def trouver_coordonnees_geographiques(region):
    geolocator = Nominatim(user_agent='my_app')
    location = geolocator.geocode(region + ', France')
    
    if location is not None:
        latitude = location.latitude
        longitude = location.longitude
        return latitude, longitude
    else:
        return None, None

In [213]:
# Utilisation de la fonction trouver_coordonnees_geographiques()
lat = []
long = []
for elt in df["Régions"]:
    latitude, longitude = trouver_coordonnees_geographiques(f"'{elt}'")

    if latitude is not None and longitude is not None:
        lat.append(latitude)
        long.append(longitude)
    else:
        print(f"Impossible de trouver les coordonnées géographiques de la région {elt}")


In [215]:
# Création du DataFrame des données avec les coordonnées géographiques des régions

new_dict_text = {
    'Administration': name_service_list,
    'Avis': new_comments_list,
    'Adresses': address_service_list,
    'Dates': new_dates_list,
    'Régions': region_list,
    'Latitudes': lat,
    'Longitudes': long
}

new_df = pd.DataFrame(new_dict_text)

In [ ]:
new_df.to_csv("./Samples/dataset_police_nationale.csv", sep=',', encoding='UTF-8', header=True, index=False)